In [32]:
from functions_v2 import*
from methods import MethodFinder

import brightway2 as bw
import bw2data as bd
import bw2analyzer as ba
import bw2calc as bc

#reduce?
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import dopo

In [21]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


In [22]:
#sector filters file names/paths

cement = 'cement_small.yaml'
electricity = 'electricity_small.yaml'
fuels= 'fuels_small.yaml'
steel = 'steel_small.yaml'
transport = 'transport_small.yaml'

In [23]:
files_dict={}
files_dict['Cement']={'yaml': 'yamls\cement_small.yaml',
                      'yaml identifier': 'Cement'}
files_dict['Electricity']= {'yaml':'yamls\electricity_small.yaml',
                            'yaml identifier': 'Electricity'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict

{'Cement': {'yaml': 'yamls\\cement_small.yaml', 'yaml identifier': 'Cement'},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity'}}

In [24]:
sectors_dict_premise = dopo.dopo_excel.process_yaml_files(files_dict=files_dict, database=ei39SSP2)

In [26]:
sectors_dict_premise

{'Cement': {'yaml': 'yamls\\cement_small.yaml',
  'yaml identifier': 'Cement',
  'activities': ['cement production, Portland' (kilogram, CH, None),
   'cement production, Portland' (kilogram, US, None),
   'cement production, Portland' (kilogram, IN, None),
   'cement production, Portland' (kilogram, CA-QC, None),
   'cement production, Portland' (kilogram, BR, None),
   'cement production, Portland' (kilogram, ZA, None),
   'cement production, Portland' (kilogram, PE, None)]},
 'Electricity': {'yaml': 'yamls\\electricity_small.yaml',
  'yaml identifier': 'Electricity',
  'activities': []}}

In [25]:
sectors_dict_ecoinvent = dopo.dopo_excel.process_yaml_files(files_dict=files_dict, database=ei39)

In [28]:
#Get Methods
finder=MethodFinder()
finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
finder.find_and_create_method(criteria=['land occupation','selected'])
finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'},
 'method_3': {'object': Brightway2 Method: selected LCI results: resource: land occupation,
  'method name': ('selected LCI results', 'resource', 'land occupation'),
  'short name': 'land occupation',
  'unit': 'square meter-year'},
 'method_4': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: use of net fresh water,
  'method na

In [29]:
lcia_method=method_dict['method_1']['method name']

In [63]:
lcia_method

('IPCC 2013', 'climate change', 'global warming potential (GWP100)')

In [72]:
lcia_method_unit=method_dict['method_1']['unit']

In [105]:
activities=sectors_dict_premise["Cement"]['activities']

In [93]:
type(activities_list)

list

In [103]:
activities_list[0].key

('ecoinvent 3.9.1 cutoff', 'df49e8f525497f2fbd56bcdc80ff0cde')

LCA Scores

In [ ]:
def compare_activities_by_grouped_leaves(
    activities,
    lcia_method,
    mode="relative",
    max_level=4,
    cutoff=7.5e-3,
    output_format="list",
    str_length=50,
):
    """Compare activities by the impact of their different inputs, aggregated by the product classification of those inputs.

    Args:
        activities: list of ``Activity`` instances.
        lcia_method: tuple. LCIA method to use when traversing supply chain graph.
        mode: str. If "relative" (default), results are returned as a fraction of total input. Otherwise, results are absolute impact per input exchange.
        max_level: int. Maximum level in supply chain to examine.
        cutoff: float. Fraction of total impact to cutoff supply chain graph traversal at.
        output_format: str. See below.
        str_length; int. If ``output_format`` is ``html``, this controls how many characters each column label can have.

    Raises:
        ValueError: ``activities`` is malformed.

    Returns:
        Depends on ``output_format``:

        * ``list``: Tuple of ``(column labels, data)``
        * ``html``: HTML string that will print nicely in Jupyter notebooks.
        * ``pandas``: a pandas ``DataFrame``.

    """
    for act in activities:
        if not isinstance(act, bd.backends.proxies.Activity):
            raise ValueError("`activities` must be an iterable of `Activity` instances")

    objs = [
        group_leaves(find_leaves(act, lcia_method, max_level=max_level, cutoff=cutoff))
        for act in activities
    ]
    sorted_keys = sorted(
        [
            (max([el[0] for obj in objs for el in obj if el[2] == key]), key)
            for key in {el[2] for obj in objs for el in obj}
        ],
        reverse=True,
    )
    name_common = commonprefix([act["name"] for act in activities])

    if " " not in name_common:
        name_common = ""
    else:
        last_space = len(name_common) - operator.indexOf(reversed(name_common), " ")
        name_common = name_common[:last_space]
        print("Omitting activity name common prefix: '{}'".format(name_common))

    product_common = commonprefix(
        [act.get("reference product", "") for act in activities]
    )

    lca = bc.LCA({act: 1 for act in activities}, lcia_method)
    lca.lci()
    lca.lcia()

    labels = [
        "activity",
        "product",
        "location",
        "unit",
        "total",
        "direct emissions",
    ] + [key for _, key in sorted_keys]
    data = []
    for act, lst in zip(activities, objs):
        lca.redo_lcia({act.id: 1})
        data.append(
            [
                act["name"].replace(name_common, ""),
                act.get("reference product", "").replace(product_common, ""),
                act.get("location", "")[:25],
                act.get("unit", ""),
                lca.score,
            ]
            + [
                (
                    lca.characterization_matrix
                    * lca.biosphere_matrix
                    * lca.demand_array
                ).sum()
            ]
            + [get_value_for_cpc(lst, key) for _, key in sorted_keys]
        )

    data.sort(key=lambda x: x[4], reverse=True)

    if mode == "relative":
        for row in data:
            for index, point in enumerate(row[5:]):
                row[index + 5] = point / row[4]

    if output_format == "list":
        return labels, data
    elif output_format == "pandas":
        return pd.DataFrame(data, columns=labels)
    elif output_format == "html":
        return tabulate.tabulate(
            data,
            [x[:str_length] for x in labels],
            tablefmt="html",
            floatfmt=".3f",
        )

In [112]:



for act in activities_list:
        if not isinstance(act, bd.backends.peewee.proxies.Activity):
            raise ValueError("`activities` must be an iterable of `Activity` instances")

data=[]

labels = [
    "activity",
    "reference product",
    "location",
    "method name",
    "method unit",
    "total",
] #+ [key for _, key in sorted_keys]

# for act in activities_list:
#     activities=[act for act in ei39SSP2 if act in activities_list]
#     print(type(act.metadata))
#     print(activities)
for act in activities:
    lca = bc.LCA({act: 1}, lcia_method)
    lca.lci()
    lca.lcia()

    data.append(
            [  #adapt this to access method better
                act["name"],
                act.get("reference product"),
                act.get("location", "")[:25],
                lcia_method,
                lcia_method_unit,
                lca.score,
            ]
    )

data_premise=pd.DataFrame(data, columns=labels)



In [113]:
data_premise

,activity,reference product,location,method name,method unit,total
0,"cement production, Portland","cement, Portland",CH,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.742421
1,"cement production, Portland","cement, Portland",US,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.885515
2,"cement production, Portland","cement, Portland",IN,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.891756
3,"cement production, Portland","cement, Portland",CA-QC,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.845772
4,"cement production, Portland","cement, Portland",BR,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.851799
5,"cement production, Portland","cement, Portland",ZA,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,1.000588
6,"cement production, Portland","cement, Portland",PE,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.895198


In [78]:
activities_ecoinvent=sectors_dict_ecoinvent['Cement']['activities'][:]

In [116]:
for act in activities_ecoinvent:
        if not isinstance(act, bd.backends.peewee.proxies.Activity):
            raise ValueError("`activities` must be an iterable of `Activity` instances")

data_ecoinvent=[]

labels = [
    "activity",
    "reference product",
    "location",
    "method name",
    "method unit",
    "total",
] #+ [key for _, key in sorted_keys]

activities_ecoinvent=[act for act in ei39SSP2 if act in activities_ecoinvent]
for act in activities_ecoinvent:
    lca = bc.LCA({act: 1} , lcia_method)
    lca.lci()
    lca.lcia()

    data.append(
            [  #adapt this to access method better
                act["name"],
                act.get("reference product"),
                act.get("location", "")[:25],
                lcia_method,
                lcia_method_unit,
                lca.score,
            ]
    )

data_ecoinvent=pd.DataFrame(data, columns=labels)



In [117]:
data_ecoinvent

,activity,reference product,location,method name,method unit,total
0,"cement production, Portland","cement, Portland",CH,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.742421
1,"cement production, Portland","cement, Portland",US,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.885515
2,"cement production, Portland","cement, Portland",IN,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.891756
3,"cement production, Portland","cement, Portland",CA-QC,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.845772
4,"cement production, Portland","cement, Portland",BR,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.851799
5,"cement production, Portland","cement, Portland",ZA,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,1.000588
6,"cement production, Portland","cement, Portland",PE,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.895198
7,"cement production, Portland","cement, Portland",CH,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.742421
8,"cement production, Portland","cement, Portland",US,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.885515
9,"cement production, Portland","cement, Portland",BR,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.851799


In [115]:
data_ecoinvent

,activity,reference product,location,method name,method unit,total
0,"cement production, Portland","cement, Portland",CH,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.742421
1,"cement production, Portland","cement, Portland",US,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.885515
2,"cement production, Portland","cement, Portland",IN,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.891756
3,"cement production, Portland","cement, Portland",CA-QC,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.845772
4,"cement production, Portland","cement, Portland",BR,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.851799
5,"cement production, Portland","cement, Portland",ZA,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,1.000588
6,"cement production, Portland","cement, Portland",PE,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.895198
7,"cement production, Portland","cement, Portland",CH,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.742421
8,"cement production, Portland","cement, Portland",US,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.885515
9,"cement production, Portland","cement, Portland",BR,"(IPCC 2013, climate change, global warming pot...",kg CO2-Eq,0.851799


In [47]:
labels

['activity', 'product', 'location', 'unit', 'total', 'direct emissions']

In [48]:
data = []
for act, lst in zip(activities, objs):
    lca.redo_lcia({act.id: 1})
    data.append(
        [
            act["name"].replace(name_common, ""),
            act.get("reference product", "").replace(product_common, ""),
            act.get("location", "")[:25],
            act.get("unit", ""),
            lca.score,
        ]
        + [
            (
                lca.characterization_matrix
                * lca.biosphere_matrix
                * lca.demand_array
            ).sum()
        ]
        + [get_value_for_cpc(lst, key) for _, key in sorted_keys]
    )

data.sort(key=lambda x: x[4], reverse=True)

NameError: name 'objs' is not defined

In [ ]:
def sector_lca_scores(main_dict, method_dict):
    '''
    Generates the LCA scores for the sectors activities in the main dictionary 
    for the different methods in the method dictionary.

    It returns the main dictionary updated as scores dictionary which also holds the former information for each sector.
    The LCA scores are stored by method name in the respective sector dictionary within the main dictionary.
    '''

    # Initialize scores_dict as a copy of main_dict
    scores_dict = main_dict.copy()

    #Loop through methods
    for method_key in method_dict.keys():
        method = method_dict[method_key]['method name']

        # Loop through each sector in main_dict
        for sector in scores_dict.keys():
            # Extract activities for the current sector
            sector_activities = scores_dict[sector]['activities']

            if method not in scores_dict[sector]:
                scores_dict[sector][method] = {}

            # Initialize a list to store activitiy scores by method
            if 'lca_score' not in scores_dict[sector][method]:
                scores_dict[sector][method]['lca_score'] = []
            
            for activity in sector_activities:
                activity_LCA = bw.LCA({activity: 1}, method)
                activity_LCA.lci()
                activity_LCA.lcia()
                score = activity_LCA.score

                # Create a tuple key with relevant information
                activity_score= {
                    'activity':activity,
                    'score':score,
                    'method':method
                    #'unit': activity.metadata
                    # activity["name"],
                    # activity["unit"],
                    # activity["location"],
                    # activity.get("reference product"),
                }

                scores_dict[sector][method]['lca_score'].append(activity_score)
                
            #ecoinvent_scores[key] = score

    return scores_dict

        # Calculate LCA scores using the specified method
    #     lca_scores = compare_activities_multiple_methods(
    #         activities_list=sector_activities,
    #         methods=method_dict,
    #         identifier=sector,
    #         mode='absolute'
    #     )
        
    #     # Apply the small_inputs_to_other_column function with the cutoff value
    #     lca_scores = small_inputs_to_other_column(lca_scores, cutoff=0.02)
        
    #     # Save the LCA scores to the scores_dict
    #     scores_dict[sector]['lca_scores'] = lca_scores

    # return scores_dict

In [ ]:
lca_premise_dict=sector_lca_scores(sectors_dict_premise, method_dict)

In [ ]:
lca_ecoinvent_dict=sector_lca_scores(sectors_dict_premise, method_dict)

In [ ]:
lca_premise_dict

In [ ]:
lca_ecoinvent_dict

In [ ]:
type(lca_ecoinvent_dict['Cement']['IPCC 2013', 'climate change', 'global warming potential (GWP100)'][0]["activitiy"])

In [ ]:
# Iterating through the dictionary to access the scores
for key in lca_ecoinvent_dict.items():
    
    for method, method_data in lca_ecoinvent_dict.items():
        # Check if the key is a tuple (representing a method)
        if isinstance(method, tuple):
            print(f"  Method: {method}")
            
            for score_entry in method_data.get('lca_score', []):
                activity = score_entry.get('activity')
                score = score_entry.get('score')
                print(f"    Activity: {activity}, Score: {score}")

                #####################################################
                ####################################################
                #####################################################

In [ ]:
def _compute_relative_change(original, transformed):
    if original == 0:
        return float("inf") if transformed != 0 else 0
    return (transformed - original) / original


def _calc_relative_changes_new(ecoinvent_dict, premise_dict):
    # Match activities_list and calculate relative changes
    relative_changes = {}
    relative_changes["method"] = ecoinvent_scores["method"]
    print(relative_changes)

    # Iterating through the dictionary to access the scores
    for key in ecoinvent_dict.items():
        
        for method, method_data in ecoinvent_dict.items():
            # Check if the key is a tuple (representing a method)
            if isinstance(method, tuple):
                print(f"  Method: {method}")
                
                for score_entry in method_data.get('lca_score', []):
                    activity = score_entry.get('activity')
                    score = score_entry.get('score')
                    print(f"    Activity: {activity}, Score: {score}")

                    #####################################################
                    ####################################################
                    #####################################################

    # Track additional keys in premise_scores
    additional_premise_keys = []

    for key, original_score in ecoinvent_scores.items():
        if (
            key in premise_scores
        ):  # activities only in premise_scores are according to this logic neglected.
            # Skip if original_score is a tuple due to information tuple key
            if isinstance(original_score, tuple):
                continue

            transformed_score = premise_scores[key]
            relative_change = _compute_relative_change(
                original_score, transformed_score
            )
            relative_changes[key] = relative_change

    # Identify additional keys in premise_scores
    for key in premise_scores.keys():
        if key not in ecoinvent_scores:
            additional_premise_keys.append(key)

    # Print the dataframes_dict
    for key, change in relative_changes.items():
        print(f"{key}: {change}")

    if additional_premise_keys:
        print(
            "Additional keys in premise_scores not found in ecoinvent_scores:",
            additional_premise_keys,
        )

    return relative_changes


In [ ]:
lca_smpl #which database is used??